# Image Classification with TensorFlow: From Classical ML to Deep Learning

This notebook covers:
- Baseline Model (Without Deep Learning)
- CNN Model from Scratch
- Transfer Learning
- Fine-tuning Pre-trained Model

In [ ]:
!pip install tensorflow scikit-learn tensorflow-datasets

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import tensorflow_datasets as tfds

## Load and preprocess CIFAR-10 dataset

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    "cifar10",
    split=["train", "test"],
    as_supervised=True,
    with_info=True,
)

def preprocess(image, label):
    image = tf.image.resize(image, (32, 32))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

BATCH_SIZE = 64

ds_train = ds_train.map(preprocess).shuffle(1000).batch(BATCH_SIZE)
ds_test = ds_test.map(preprocess).batch(BATCH_SIZE)

## 1. Baseline Model (Without Deep Learning)

In [ ]:
X_train, y_train = [], []
for img, label in tfds.as_numpy(ds_train):
    X_train.append(img.reshape(-1))
    y_train.append(label)

X_test, y_test = [], []
for img, label in tfds.as_numpy(ds_test):
    X_test.append(img.reshape(-1))
    y_test.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

baseline_model = LogisticRegression(max_iter=100)
baseline_model.fit(X_train, y_train)

y_pred = baseline_model.predict(X_test)

print("Baseline Accuracy:", accuracy_score(y_test, y_pred))

## 2. CNN Model from Scratch

In [ ]:
cnn_model = Sequential([
    layers.Input(shape=(32, 32, 3)),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

cnn_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

cnn_model.fit(ds_train, epochs=10, validation_data=ds_test)

## 3. Transfer Learning (Feature Extraction)

In [ ]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(96, 96, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

def preprocess_mobilenet(image, label):
    image = tf.image.resize(image, (96, 96))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

ds_train_mobilenet = ds_train.map(preprocess_mobilenet)
ds_test_mobilenet = ds_test.map(preprocess_mobilenet)

model_transfer = Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model_transfer.compile(optimizer='adam',
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])

model_transfer.fit(ds_train_mobilenet, epochs=5, validation_data=ds_test_mobilenet)

## 4. Fine-Tuning the Pre-trained Model

In [ ]:
base_model.trainable = True

model_transfer.compile(optimizer=keras.optimizers.Adam(1e-5),
                       loss='sparse_categorical_crossentropy',
                       metrics=['accuracy'])

model_transfer.fit(ds_train_mobilenet, epochs=5, validation_data=ds_test_mobilenet)

## Results Comparison

In [ ]:
cnn_loss, cnn_acc = cnn_model.evaluate(ds_test)
transfer_loss, transfer_acc = model_transfer.evaluate(ds_test_mobilenet)

print(f"Baseline Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print(f"CNN Accuracy: {cnn_acc:.2f}")
print(f"Transfer Learning Accuracy: {transfer_acc:.2f}")